In [1]:
import os
import numpy as np
import json
import pandas as pd
import itertools
from tqdm import tqdm
import nltk
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import re

In [2]:
def create_dataframe(dataset_path, source):
    """Function used to create a Pandas DataFrame containing specifications page titles

    Reads products specifications from the file system ("dataset_path" variable in the main function) and creates a Pandas DataFrame where each row is a
    specification. The columns are 'source' (e.g. www.sourceA.com), 'spec_number' (e.g. 1) and the 'page title'. Note that this script will consider only
    the page title attribute for simplicity.

    Args:
        dataset_path (str): The path to the dataset

    Returns:
        df (pd.DataFrame): The Pandas DataFrame containing specifications and page titles
    """

    print('>>> Creating dataframe...\n')
    columns_df = ['source', 'spec_number', 'spec_id', 'page_title']
    
    df = pd.DataFrame()
    progressive_id = 0
    progressive_id2row_df = {}
    #for source in tqdm(os.listdir(dataset_path)):
    #source = "buy.net"
    for specification in os.listdir(os.path.join(dataset_path, source)):
        specification_number = specification.replace('.json', '')
        specification_id = '{}//{}'.format(source, specification_number)
        with open(os.path.join(dataset_path, source, specification)) as specification_file:
            columns_df = ['source', 'spec_number', 'spec_id']
            specification_data = json.load(specification_file)
            attrs = []
            for k, v in specification_data.items():
                columns_df.append(k)
                attrs.append(v)
            row = [source, specification_number, specification_id]
            row.extend(attrs)
            progressive_id2row_df.update({progressive_id: row})
            progressive_id += 1
            df = df.append(pd.DataFrame.from_dict(progressive_id2row_df, orient='index', columns=columns_df))
            progressive_id2row_df = {}
    #print(df)
    print('>>> Dataframe created successfully!\n')
    return df

In [3]:
df = create_dataframe('../../datasets/unlabeled/2013_camera_specs', "www.buzzillions.com")

>>> Creating dataframe...



//anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


>>> Dataframe created successfully!



In [4]:
cols = ['spec_id',"megapixels"]

In [5]:
df = df[cols]

In [6]:
df.head()

,spec_id,megapixels
0,www.buzzillions.com//2560,16
1,www.buzzillions.com//2075,7.0
2,www.buzzillions.com//2425,16 Megapixels
3,www.buzzillions.com//1966,10 Megapixels
4,www.buzzillions.com//953,14


### megapixels

In [7]:
df[df['megapixels'].notnull()]

,spec_id,megapixels
0,www.buzzillions.com//2560,16
1,www.buzzillions.com//2075,7.0
2,www.buzzillions.com//2425,16 Megapixels
3,www.buzzillions.com//1966,10 Megapixels
4,www.buzzillions.com//953,14
5,www.buzzillions.com//2219,12.1
6,www.buzzillions.com//2537,14
7,www.buzzillions.com//2472,14
8,www.buzzillions.com//2022,"[12.0, 12.1]"
9,www.buzzillions.com//1874,14


In [8]:
def parse_megapixels(value):
    if(isinstance(value, list)): #In case of a list keep only the first one
        value=value[0]
    if pd.isna(value):
        return value
    else:
        match1 = re.search('(\d*\.\d+|\d+)( Megapixels|)', str(value))
        if match1 is None: 
            print(value)
            return float("NaN")


        return (float(match1.group(1).replace(",",".")))

        

In [9]:
df["megapixels"] = df["megapixels"].apply(parse_megapixels)


In [10]:
df.head()

,spec_id,megapixels
0,www.buzzillions.com//2560,16.0
1,www.buzzillions.com//2075,7.0
2,www.buzzillions.com//2425,16.0
3,www.buzzillions.com//1966,10.0
4,www.buzzillions.com//953,14.0


### Final cleaning

In [11]:
df.to_csv("../../datasets/unlabeled/cleaned/buzzillions.csv", index=False)
